# Training / Fine-tuning a Text-to-Phon model

We are going to look at model fine-tuning by taking a general purpose language model and fine-tuning it to translate text to IPA

In [ ]:
!pip install accelerate -U
!pip install transformers -U
!pip install datasets

In [ ]:
import transformers
import datasets
from datasets import load_dataset
import accelerate


In [ ]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("phonemetransformers/IPA-BabyLM", "strict_small")

### Fine-Tuning

To fine tune BART from scratch uncomment the next five blocks of code and run. Note though that it will take a good few hours to run for each epoch.

In [ ]:
#from transformers import AutoTokenizer, BartForConditionalGeneration
#device="cuda"
#tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
#model = BartForConditionalGeneration.from_pretrained("facebook/bart-base").to(device)

In [ ]:
#tokenset = list(set(str.split(' '.join(ds["train"][:]["phonemized_utterance"]))))

#print(len(tokenizer))  # 28996
#tokenizer.add_tokens(tokenset)
#print(len(tokenizer))  # 28997

#model.resize_token_embeddings(len(tokenizer))


#def convert_examples_to_features(example_batch):
#    input_encodings = tokenizer(example_batch["text"], max_length=1024,
#                                truncation=True)

#    with tokenizer.as_target_tokenizer():
#        target_encodings = tokenizer(example_batch["phonemized_utterance"], max_length=1024,
#                                     truncation=True)

#    return {"input_ids": input_encodings["input_ids"],
#            "attention_mask": input_encodings["attention_mask"],
#            "labels": target_encodings["input_ids"]}

#ds_pt = ds.map(convert_examples_to_features, batched=True)
#columns = ["input_ids", "labels", "attention_mask"]
#ds_pt.set_format(type="torch", columns=columns)

#from transformers import DataCollatorForSeq2Seq, TrainingArguments, Trainer

#seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

#training_args = TrainingArguments(
#    output_dir='text-to-phon', num_train_epochs=1, warmup_steps=500,
#    per_device_train_batch_size=1, per_device_eval_batch_size=1,
#    weight_decay=0.01, logging_steps=10, push_to_hub=False,
#    evaluation_strategy='steps', eval_steps=2500, save_steps=1e6,gradient_accumulation_steps=128)

#trainer = Trainer(model=model, args=training_args,
#                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
#                  train_dataset=ds_pt["train"],
#                  eval_dataset=ds_pt["valid"])


In [ ]:
#!pip install wandb

In [ ]:
#import wandb
#from huggingface_hub import notebook_login

#notebook_login()
#wandb.init(mode="disabled")

In [ ]:
# hide_output
#torch.cuda.empty_cache()
#trainer.train()
# To save your fine-tuned model:
#trainer.save_model("dialogue-summ-model-bart")

To use a pre-tuned (for one-epoch only so performance isn't great yet) model run the following

In [ ]:
!gdown 1VAJMdR2kfHQkJRYQToLT8oHXEYfKJbZP

!gunzip bart-text-to-phon.tar.gz
!tar xf bart-text-to-phon.tar

In [ ]:
from transformers import AutoTokenizer, BartForConditionalGeneration
model_ckpt="bart-text-to-phon"
device="cuda"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = BartForConditionalGeneration.from_pretrained(model_ckpt).to(device)

### Translating new input to IPA

In [ ]:
input_utterance = "mummy"
input_ = tokenizer(input_utterance, max_length=1024, truncation=True, return_tensors="pt")
device="cuda"
input_ids = input_['input_ids']
input_mask = input_['attention_mask']
responses = model.generate(input_ids=input_ids.to(device),
                         attention_mask=input_mask.to(device),
                         num_beams=100,
                         no_repeat_ngram_size=2,
                         early_stopping=True,
                         num_return_sequences=1,
                         max_length=1024)
tokenizer.batch_decode(responses, skip_special_tokens=True)